**This Notebook is for finding the intersection of songs that are in the original songs playlist (13k) and the playlists (1mil)**

First Dataset: 13 thousand Spotify Songs with lyrics

In [1]:
import pandas as pd

songs_playlist = pd.read_csv("data/spotify_songs.csv")

songs_playlist.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.01170,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en


In [2]:
original_songs = []

for song in songs_playlist["track_name"]: #collecting only track_names into list
    original_songs.append(song)

In [47]:
original_songs[100] #the list works

'Nobody (feat. Athena Cage) - Single Version; 2007 Remaster'

Second Dataset: 1 million spotify playlists

In [3]:
import pandas as pd
import json

data = json.load(open('data/mpd.slice.0-999.json'))

playlists_dataset = pd.DataFrame(data["playlists"])
playlists_dataset.head()

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37,NaN
1,Awesome Playlist,false,1,1506556800,39,23,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",5,11656470,21,NaN
2,korean,false,2,1505692800,64,51,1,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",18,14039958,31,NaN
3,mat,false,3,1501027200,126,107,1,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",4,28926058,86,NaN
4,90s,false,4,1401667200,17,16,2,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",7,4335282,16,NaN


In [ ]:
init = 0
fin = 999

eval_playlists = {}

for i in range(1000):
    
    name = str(init) + "-" + str(fin) + ".json" #each document has its own extension
    
    data = json.load(open('data/mpd.slice.' + name)) #there are 1k documents with 1k playlists each

    init += 1000
    fin += 1000
    
    playlists_dataset = pd.DataFrame(data["playlists"])

    for ind, playlist in enumerate(playlists_dataset["name"]): 
        
        count = 0
        matched_songs = []
                
        for song in playlists_dataset["tracks"].iloc[ind]: 
                        
            if song.get('track_name') in original_songs: 
            
                    matched_songs.append(song.get('track_name'))
                    count += 1
            
        if count > 9: #filter out playlists with length of less than 10.
            
            eval_playlists[playlist] = matched_songs

Writing file to csv did not allow easy access:

In [28]:
import csv

file = open("data/overlap_playlists.csv", "w")

writer = csv.writer(file)

writer.writerow(["playlist", "songs"])

for key, value in eval_playlists.items():
    t = ""
    for val in value:
        t += val+"***"
    writer.writerow([key, t])

file.close()

In [34]:
overlap_playlists = pd.read_csv("data/overlap_playlists.csv")

overlap_playlists.head()

,playlist,songs
0,Throwbacks,"['Down', 'Halo', ""I'm Yours"", ""Really Don't Ca..."
1,Awesome Playlist,"['Somebody That I Used To Know', 'One More Tim..."
2,mat,"['Congratulations', 'Young Dumb & Broke', 'Sel..."
3,90s,"['Wonderwall - Remastered', 'Zombie', 'Califor..."
4,Wedding,"['I Want It That Way', 'Jealous - Remix', 'Bea..."


In [29]:
len(eval_playlists) #should be big enough for evaluation

73107

Writing Data as Dict into Json for easy access:

In [37]:
overlapped = open("data/data.json", "w")
json.dump(eval_playlists, a_file)
overlapped.close()

In [45]:
overlapped = open("data/data.json", "r")
output = overlapped.read()

print(type(output))
res = json.loads(output)

print(type(res))

for key, val in res.items():
    for song in val:
        if song not in original_songs: #double checking previous algorithm
            print(song) #since no songs printed, algorithm worked

<class 'str'>
<class 'dict'>
